# Worksheet 08

Name: Dimitriy Shchedrovitskiy
UID: U62683393

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [14]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.9019154247726595, 4.332661288238307, 4.32175607487372, 5.964466513868713, 5.724344276866726, 2.582138658256576, 4.576501498593243, 4.64508217998593, 5.3027369597967, 4.725254468688882]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [15]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[6.513960829631813, 7.167737299186551, 8.006261175029167, 7.3847967058766635, 9.688519394542503, 7.849903791125588, 9.515376488526375, 6.493323359070477, 5.55782619822252, 8.807481027191407]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [16]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.807481027191407, 5.55782619822252, 6.493323359070477, 9.515376488526375, 4.725254468688882, 5.3027369597967, 7.849903791125588, 4.64508217998593, 9.688519394542503, 7.3847967058766635]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [17]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1) / len(s1), sum(s2) / len(s2)]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) ,sum(map(lambda x: (x - mean[1]) ** 2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.55782619822252, 6.493323359070477, 4.725254468688882, 5.3027369597967, 4.64508217998593]
[8.807481027191407, 9.515376488526375, 7.849903791125588, 9.688519394542503, 7.3847967058766635]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.344844633152902,  mean_2 = 8.649215481452506
var_1 = 0.44793940997260845,  var_2 = 0.8186179030458838


c:\Users\mitia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


mean_1 = 6.004173720056823,  mean_2 = 8.257317905142962
var_1 = 0.5069024003593422,  var_2 = 0.75913354141348



e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [18]:
from scipy.stats import norm

prob_s0_x = []
prob_s1_x = [] 
prob_x = [] 

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        prob_s[j]

    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]
    prob_s0_x_i = (pdf_i[0] * prob_s[0]) / prob_x
    prob_s1_x_i = (pdf_i[1] * prob_s[1]) / prob_x
    prob_s0_x.append( prob_s0_x_i )
    prob_s1_x.append( prob_s1_x_i )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()
cluster_assignments = []
for p in probs:
    if p[1] > p[2]:
        cluster_assignments.append("S_0")
    else:
        cluster_assignments.append("S_1")

true_clusters = [] 

for i, assignment in enumerate(cluster_assignments):
    print(f"Data point {i}: Estimated Cluster = {assignment}")
    if true_clusters:
        print(f"Data point {i}: True Cluster = {true_clusters[i]}")
    print()


point =  8.807481027191407
probability of observing that point if it came from cluster 0 =  9.419505537178216e-14
probability of observing that point if it came from cluster 1 =  0.4783132436106769
point =  5.55782619822252
probability of observing that point if it came from cluster 0 =  0.7954264486809993
probability of observing that point if it came from cluster 1 =  0.0003900570414624683
point =  6.493323359070477
probability of observing that point if it came from cluster 0 =  0.03328420979887263
probability of observing that point if it came from cluster 1 =  0.015196944856216487
point =  9.515376488526375
probability of observing that point if it came from cluster 0 =  1.3374129738718113e-19
probability of observing that point if it came from cluster 1 =  0.278436721498346
point =  4.725254468688882
probability of observing that point if it came from cluster 0 =  0.34216455508392035
probability of observing that point if it came from cluster 1 =  4.994713516588068e-06
point =  5

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [19]:
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
    sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.268085811571965,  mean_2 = 8.521701140524259
var_1 = 0.38398518950660987,  var_2 = 1.029344975758878


Variance for 1 decreased and increased for var 2

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [20]:
prob_s0_x = []
prob_s1_x = []

for p in data:
    pdf_i = []

    for j in range(k):
        pdf_ij = norm.pdf(p, mean[j], var[j])
        pdf_i.append(pdf_ij)

    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_s0_x_i = (pdf_i[0] * prob_c[0]) / prob_x_i
    prob_s1_x_i = (pdf_i[1] * prob_c[1]) / prob_x_i

    prob_s0_x.append(prob_s0_x_i)
    prob_s1_x.append(prob_s1_x_i)

print(prob_s0_x_i, prob_s1_x_i)

1.0964554772279177e-06 0.9999989035445227


Its now much more confident

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [21]:
threshold = 0.99

hard_assignments = []

for p_s0_x, p_s1_x in zip(prob_s0_x, prob_s1_x):
    if p_s0_x > threshold:
        hard_assignments.append(0)  
    else:
        hard_assignments.append(1) 

for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} is assigned to cluster {assignment}")


Data point 0 is assigned to cluster 1
Data point 1 is assigned to cluster 0
Data point 2 is assigned to cluster 1
Data point 3 is assigned to cluster 1
Data point 4 is assigned to cluster 0
Data point 5 is assigned to cluster 0
Data point 6 is assigned to cluster 1
Data point 7 is assigned to cluster 0
Data point 8 is assigned to cluster 1
Data point 9 is assigned to cluster 1
